In [1]:
import numpy as np
import pandas as pd
import ast
import pickle

import gensim
from gensim import corpora
from gensim.models import CoherenceModel
import pyLDAvis.gensim

C:\Users\USER\Anaconda3\lib\site-packages\gensim\utils.py:1212: UserWarning: detected Windows; aliasing chunkize to chunkize_serial
  warnings.warn("detected Windows; aliasing chunkize to chunkize_serial")


In [2]:
data = pd.read_csv("clean.csv")
data.head()

,id,user_name,created_at,text,trendings,clean
0,1123983568500400129,me_idy,2019-05-02 16:12:19,#SemogaPuasaTahunIni nggak sendiri lagi.. Tapi...,#SemogaPuasaTahunIni,"['#semogapuasatahunini', 'ekspetasi', 'nyata',..."
1,1123983419283845125,robbseptiants,2019-05-02 16:11:44,#SemogaPuasaTahunIni bisa tahan godaan dari or...,#SemogaPuasaTahunIni,"['#semogapuasatahunini', 'tahan', 'goda', 'ora..."
2,1123983321787195393,natamiayunda,2019-05-02 16:11:21,RT @gryprmna: #SemogaPuasaTahunIni kaga mikiri...,#SemogaPuasaTahunIni,"['#semogapuasatahunini', 'pikir', 'orang', 'te..."
3,1123983291210539008,AlFathirAU__,2019-05-02 16:11:13,#SemogaPuasaTahunIni disehatkan mental dan jas...,#SemogaPuasaTahunIni,"['#semogapuasatahunini', 'disehatkan', 'mental..."
4,1123983249364090881,Feriikrnwn_,2019-05-02 16:11:03,#SemogaPuasaTahunIni bukber nya ga wacana semuaa,#SemogaPuasaTahunIni,"['#semogapuasatahunini', 'bukber', 'wacana', '..."


In [3]:
mylist = list()

for i in data['clean']:
    data = ast.literal_eval(i)
    mylist.append(data)

In [31]:
# Create Dictionary
dictionary = corpora.Dictionary(mylist)

# Create Corpus
texts = mylist

# Term Document Frequency
corpus = [dictionary.doc2bow(text) for text in mylist]

# View
print([[(dictionary[id], freq) for id, freq in cp] for cp in corpus[:1]])
print('\n')
for cp in corpus[:1]:
    for id, freq in cp:
        print(dictionary[id],freq)

# pickle.dump(corpus, open('corpus.pkl', 'wb'))
# dictionary.save('dictionary.gensim')

[[('#semogapuasatahunini', 1), ('ekspetasi', 1), ('jomblo', 1), ('nyata', 1)]]


#semogapuasatahunini 1
ekspetasi 1
jomblo 1
nyata 1


In [34]:
print('corpus\n')
corpus[:1]

corpus



[[(0, 1), (1, 1), (2, 1), (3, 1)]]

In [11]:
# Build LDA model
NUM_TOPICS = 3
lda_model = gensim.models.ldamodel.LdaModel(corpus=corpus,
                                           id2word=dictionary,
                                           num_topics=NUM_TOPICS, 
                                           random_state=500,
                                           update_every=1,
                                           chunksize=500,
                                           passes=500,
                                           alpha='auto',
                                           per_word_topics=True)

In [35]:
# Print the Keyword in the 10 topics
topics = lda_model.print_topics(num_words=5)
for topic in topics:
    
    print(topic)

(0, '0.153*"#alasantetaphidup" + 0.021*"orang" + 0.021*"hidup" + 0.014*"mati" + 0.007*"lihat"')
(1, '0.083*"demokrat" + 0.081*"partai" + 0.025*"komandan" + 0.024*"temu" + 0.024*"presiden"')
(2, '0.136*"#semogapuasatahunini" + 0.016*"puasa" + 0.015*"bukber" + 0.014*"penuh" + 0.012*"teman"')


In [13]:
# Compute Perplexity
print('\nPerplexity: ', lda_model.log_perplexity(corpus))  # a measure of how good the model is. lower the better.

# Compute Coherence Score
coherence_model_lda = CoherenceModel(model=lda_model, texts=mylist, dictionary=dictionary, coherence='c_v')
coherence_lda = coherence_model_lda.get_coherence()
print('\nCoherence Score: ', coherence_lda)


Perplexity:  -6.355830160523258

Coherence Score:  0.3679978907601755


In [14]:
# Visualize the topics
pyLDAvis.enable_notebook()
vis = pyLDAvis.gensim.prepare(lda_model, corpus, dictionary)
vis

C:\Users\USER\Anaconda3\lib\site-packages\pyLDAvis\_prepare.py:257: FutureWarning: Sorting because non-concatenation axis is not aligned. A future version
of pandas will change to not sort by default.

To accept the future behavior, pass 'sort=False'.

To retain the current behavior and silence the warning, pass 'sort=True'.

  return pd.concat([default_term_info] + list(topic_dfs))


PreparedData(topic_coordinates=              x         y  topics  cluster       Freq
topic                                                
1     -0.291100 -0.008306       1        1  40.918270
2      0.155168 -0.184393       2        1  32.083256
0      0.135932  0.192699       3        1  26.998476, topic_info=     Category        Freq                       Term       Total  loglift  \
22    Default  741.000000          #alasantetaphidup  741.000000  30.0000   
0     Default  783.000000       #semogapuasatahunini  783.000000  29.0000   
2631  Default  610.000000                   demokrat  610.000000  28.0000   
2633  Default  595.000000                     partai  595.000000  27.0000   
2660  Default  185.000000                   komandan  185.000000  26.0000   
34    Default  174.000000                   presiden  174.000000  25.0000   
2659  Default  172.000000                    kogasma  172.000000  24.0000   
2668  Default  171.000000                        ahy  171.000000  23.0000   
2689  Default  169.000000                      pileg  169.000000  22.0000   
426   Default  190.000000                       temu  190.000000  21.0000   
2657  Default  130.000000                     istana  130.000000  20.0000   
59    Default   93.000000                      puasa   93.000000  19.0000   
9     Default  168.000000                      orang  168.000000  18.0000   
19    Default   86.000000                     bukber   86.000000  17.0000   
31    Default   80.000000                      penuh   80.000000  16.0000   
1388  Default   67.000000                       mati   67.000000  15.0000   
1205  Default   98.000000                     jokowi   98.000000  14.0000   
184   Default  132.000000                      hidup  132.000000  13.0000   
2655  Default   95.000000                       agus   95.000000  12.0000   
2565  Default   93.000000                      kursi   93.000000  11.0000   
79    Default   71.000000                      teman   71.000000  10.0000   
2632  Default   91.000000                      kader   91.000000   9.0000   
225   Default   91.000000                      hasil   91.000000   8.0000   
2656  Default   91.000000                  harimurti   91.000000   7.0000   
143   Default   69.000000                     lancar   69.000000   6.0000   
169   Default   86.000000                       jawa   86.000000   5.0000   
2647  Default   84.000000                       aneh   84.000000   4.0000   
2688  Default   84.000000                      kejut   84.000000   3.0000   
2690  Default   82.000000                        sud   82.000000   2.0000   
2662  Default   78.000000                  yudhoyono   78.000000   1.0000   
...       ...         ...                        ...         ...      ...   
848    Topic3   23.836803                       dosa   24.387522   1.2865   
1443   Topic3   19.429680                   malaikat   19.970753   1.2819   
1248   Topic3   19.358282                    because   19.899879   1.2818   
1895   Topic3   17.910515                      tuhan   18.454233   1.2795   
1429   Topic3   16.592619                       esok   17.133692   1.2773   
1546   Topic3   17.036907                      gagal   17.601664   1.2768   
490    Topic3   16.418974                     hadiah   17.035910   1.2725   
1431   Topic3   14.263206                       teta   14.804278   1.2722   
1456   Topic3   13.921864                      paham   14.462936   1.2713   
1579   Topic3   13.849341                     izrail   14.390414   1.2711   
227    Topic3   13.075883                       ortu   13.618591   1.2687   
1254   Topic3   13.712786                      utang   14.286869   1.2684   
1599   Topic3   12.260139                        cod   12.801211   1.2662   
2040   Topic3   11.519928                        our   12.061001   1.2635   
24     Topic3   11.595765  #diskualifikasiyangcurang   12.142932   1.2633   
403    Topic3   11.198341                  bismillah   11.764326   1

In [ ]:
new_doc = 'Practical Bayesian Optimization of Machine Learning Algorithms'
new_doc = prepare_text_for_lda(new_doc)
new_doc_bow = dictionary.doc2bow(new_doc)
print(new_doc_bow)
print(ldamodel.get_document_topics(new_doc_bow))